# Basic AzureML SDK V2 (Preview) functionality

In [1]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient


credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<Subscription ID>"
    resource_group = "<ResourceGroup Name>"
    workspace = "<<WorkspaceName>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
print(ml_client)

##Save the Workspace Config ??? Documentation points to Workspace.write_config but could not find any definition for the same

We could not find config.json in: . or in its parent directories. 
MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f966bf2bfd0>,
         subscription_id=25758354-2fe7-426a-be0e-1ad20058340d,
         resource_group_name=azureml-live,
         workspace_name=ftaliveazureml)


# Computes

In [2]:
#List Compute targets in the Workspace
for compute in ml_client.compute.list():
    print(f"Compute {compute.name} is a {compute.type}")

Compute ftaliveciw3622 is a computeinstance
Compute cpu-cluster is a amlcompute
Compute cpu-sm-cluster is a amlcompute
Compute varmag3-ci is a computeinstance


In [8]:
from azure.ai.ml.entities import AmlCompute

# provision a small compute cluster
cpu_compute_target = "cpu-sm-cluster"

try:
    cluster = ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="STANDARD_D2_V2", min_instances=0, max_instances=4
    )
    cluster = ml_client.compute.begin_create_or_update(compute)

print(f"Got reference to cluster: {cluster.name}, Type: {cluster.type}")

Refer for more examples on Compute: https://github.com/Azure/azureml-examples/blob/main/sdk/resources/compute/compute.ipynb

# Datastores

In [7]:
#List existing Datastorses in the Workspace and select the default store

print([dstore.name for dstore in ml_client.datastores.list()])

dstore_name = "workspaceblobstore"

dstore = ml_client.datastores.get(dstore_name)
# OR 
dstore = ml_client.datastores.get_default()

print(f"Default Datastore name: {dstore.name}, Type: {dstore.type}")

['azureml_globaldatasets', 'workspaceartifactstore', 'workspaceworkingdirectory', 'workspacefilestore', 'workspaceblobstore']
Default Datastore name: workspaceblobstore, Type: AzureBlob


In [ ]:
#Create a Datastore
'''
blob_datastore1 = AzureBlobDatastore(
    name="blob-example",
    description="Datastore pointing to a blob container.",
    account_name="mytestblobstore",
    container_name="data-container",
    credentials={
        "account_key": "XXXxxxXXXxXXXXxxXXXXXxXXXXXxXxxXxXXXxXXXxXXxxxXXxxXXXxXxXXXxxXxxXXXXxxxxxXXxxxxxxXXXxXXX"
    },
)
ml_client.create_or_update(blob_datastore1)'''
# ref: https://github.com/Azure/azureml-examples/blob/main/sdk/resources/datastores/datastore.ipynb

Refer for more examples on Datastores: https://github.com/Azure/azureml-examples/blob/main/sdk/resources/datastores/datastore.ipynb

# Data Assets

In [20]:
#Data Assets
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# Supported paths include:
# local: './<path>'
# blob:  'https://<account_name>.blob.core.windows.net/<container_name>/<path>'
# ADLS gen2: 'abfss://<file_system>@<account_name>.dfs.core.windows.net/<path>/'
# Datastore: 'azureml://datastores/<data_store_name>/paths/<path>'

#E.g Datastore example 
# data_uri_path = 'azureml://datastores/workspaceblobstore/paths/samples/diabetes/v1/diabetes_raw_data.csv' (file needs to exist in the location)

#Blob/http location 
data_uri_path = 'https://azuremlexamples.blob.core.windows.net/datasets/diabetes.csv'

file_data_asset = Data(
    path= data_uri_path,
    type=AssetTypes.URI_FILE, ##URI_FILE/URI_FOLDER/MLTABLE
    description="Diabetes Sample Dataset",
    name="diabetes-dataset-uri-file",
    #version='1'
)

#creates and registers the data asset in the workspace
ml_client.data.create_or_update(file_data_asset)

diabetes_uri_data = ml_client.data.get('diabetes-dataset-uri-file', version="3")
print(diabetes_uri_data.id, diabetes_uri_data.path)

/subscriptions/25758354-2fe7-426a-be0e-1ad20058340d/resourceGroups/azureml-live/providers/Microsoft.MachineLearningServices/workspaces/ftaliveazureml/data/diabetes-dataset-uri-file/versions/3 https://azuremlexamples.blob.core.windows.net/datasets/diabetes.csv


# MLTable
mltable is a way to abstract the schema definition for tabular data so that it is easier for consumers of the data to materialize the table into a Pandas/Dask/Spark dataframe.

In [22]:
#mltable

mltable_path = 'azureml://datastores/workspaceblobstore/paths/samples/diabetes_mltable/'
#Upload the sample MLTable.yml file and .csv file from 'sample_data' folder in to Blobstore location


mltable_asset = Data(
    path=mltable_path,
    type=AssetTypes.MLTABLE,
    description="Sklearn, Diabetes Sample Dataset",
    name="diabetes-mltable",
    #version='1'
)

diabetes_mltable = ml_client.data.create_or_update(mltable_asset)

print(diabetes_mltable.id, diabetes_mltable.path)

/subscriptions/25758354-2fe7-426a-be0e-1ad20058340d/resourceGroups/azureml-live/providers/Microsoft.MachineLearningServices/workspaces/ftaliveazureml/data/diabetes-mltable/versions/1 azureml://subscriptions/25758354-2fe7-426a-be0e-1ad20058340d/resourcegroups/azureml-live/workspaces/ftaliveazureml/datastores/workspaceblobstore/paths/samples/diabetes_mltable/


In [25]:
#Reading MLTable Data using mltable package 
#!pip install mltable

from mltable import load

tbl = load(uri= "./sample_data")
df = tbl.to_pandas_dataframe()
df.head()

,0,1,2,3,4,5,6,7,8,9,target
0,0.0380759064334241,0.0506801187398187,0.0616962065186885,0.0218723549949558,-0.0442234984244464,-0.0348207628376986,-0.0434008456520269,-0.00259226199818282,0.0199084208763183,-0.0176461251598052,151.0
1,-0.00188201652779104,-0.044641636506989,-0.0514740612388061,-0.0263278347173518,-0.00844872411121698,-0.019163339748222,0.0744115640787594,-0.0394933828740919,-0.0683297436244215,-0.09220404962683,75.0
2,0.0852989062966783,0.0506801187398187,0.0444512133365941,-0.00567061055493425,-0.0455994512826475,-0.0341944659141195,-0.0323559322397657,-0.00259226199818282,0.00286377051894013,-0.0259303389894746,141.0
3,-0.0890629393522603,-0.044641636506989,-0.0115950145052127,-0.0366564467985606,0.0121905687618,0.0249905933641021,-0.0360375700438527,0.0343088588777263,0.0226920225667445,-0.0093619113301358,206.0
4,0.00538306037424807,-0.044641636506989,-0.0363846922044735,0.0218723549949558,0.00393485161259318,0.0155961395104161,0.0081420836051921,-0.00259226199818282,-0.0319914449413559,-0.0466408735636482,135.0


Refer for more on handling data assets here: https://github.com/Azure/azureml-examples/blob/main/sdk/assets/data/data.ipynb